In [ ]:
from google.colab import drive
drive.mount('/content/Mydrive') 

Mounted at /content/Mydrive


In [ ]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from numpy import sqrt

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Load data

In [ ]:
#path = "/content/drive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering"
train = pd.read_csv("/content/Mydrive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_reviews_train_than30.csv")
test = pd.read_csv("/content/Mydrive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_reviews_test_than30.csv")

book_information = pd.read_csv("/content/Mydrive/MyDrive/recomment_system/Do_an/data_final/collaborative-fittering/final_books_than30.csv")

In [ ]:
train.head()

,id,product_id,user_id,user_name,rating,content,thank_count,rating_average,count
0,6081499,3734909.0,88616.0,Trương Minh Đăng,5,NaN,0,4.8,1
1,12731297,14961285.0,16848626.0,Phạm Văn Tùng,5,NaN,0,4.7,1
2,7849857,68774130.0,8046945.0,Minh Giải,5,NaN,0,5.0,1
3,15666018,152449148.0,1333575.0,Genny Lee,5,bộ truyện rất hay và giàu tính nhân văn.,0,4.9,1
4,17241576,655679.0,24504222.0,Nhật Việt,5,NaN,0,4.9,1


In [ ]:
train.shape

(140839, 9)

In [ ]:
test.shape

(1423, 9)

In [ ]:
book_information.shape

(16587, 17)

In [ ]:
book_information.head(3)

,category,url,id,sku,name,price,rating_average,review_count,all_time_quantity_sold,description,specifications,seller_id,seller_sku,seller_name,seller_store_id,count_specification,attributes
0,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p141483251.html?spid=1...,141483251.0,8.070578e+12,Sách: Thai Giáo Theo Chuyên Gia - 280 Ngày Mỗi...,80500.0,5.0,58.0,508.0,"\nThai Giáo Theo Chuyên Gia, 280 Ngày, Mỗi Ngà...","{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",36846.0,3.544877e+12,Minh Long Book,36125.0,5,"{'id': 141483251.0, 'publisher_vn': 'Minh Long..."
1,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p125362334.html?spid=1...,125362334.0,8.806987e+12,Dinh Dưỡng Thai Kỳ (Bà Bầu Ăn Gì Giúp Con Phát...,103950.0,4.8,41.0,486.0,“Dinh dưỡng thai kỳ” giống như một cuốn bách k...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",1.0,6.946825e+12,Tiki Trading,40395.0,7,"{'id': 125362334.0, 'publisher_vn': 'Booklife'..."
2,Nhà sách tiki/sách tiếng việt/sách bà mẹ em bé...,https://tiki.vn/product-p58213080.html?spid=58...,58213080.0,4.467229e+12,Montessori – Phương Pháp Giáo Dục Toàn Diện Ch...,95200.0,5.0,16.0,116.0,MONTESSORI – PHƯƠNG PHÁP GIÁO DỤC TOÀN DIỆN CH...,"{""name"": ""Th\u00f4ng tin chung"", ""attributes"":...",53660.0,1.012848e+12,Nhà sách Fahasa,51001.0,6,"{'id': 58213080.0, 'publisher_vn': '1980 Books..."


# Một vài hàm lấy dữ liệu

In [ ]:
def get_rating(userid, bookid):
    return (train.loc[((train.user_id== userid)&(train.product_id== bookid)), 'rating'].iloc[0])
#------------------------
def get_bookid(userid): 
    return (train.loc[(train.user_id==userid), 'product_id'].tolist())
#----------------------
def get_userid(bookid): 
    return (train.loc[(train.product_id==bookid), 'user_id'].tolist())
#------------------------
def get_book_title(bookid):
    return (book_information.loc[(book_information.id==bookid), 'name'].iloc[0])
#------------------------
def get_average_rating_user(userid):
    rating_user = train.loc[((train.user_id== userid)), 'rating']
    return np.mean(rating_user)
#-----------------------
def get_average_rating_book(bookid):
    rating_book = train.loc[((train.product_id== bookid)), 'rating']
    return np.mean(rating_book)

In [ ]:
get_average_rating_book(58543601.0)

4.826086956521739

In [ ]:
average_rating_bookid = {}
for i in train["product_id"].unique():
    average_rating_bookid[i]=get_average_rating_book(i)

# Các hàm tính độ tương tự 

## Độ đo Cosine

In [ ]:
def cosine_similarity_user(user1, user2):
    both_watch_count = 0
    for bookid in train.loc[train.user_id == user1, 'product_id'].to_list():
        for bookid in train.loc[train.user_id == user2, 'product_id'].tolist():
            both_watch_count +=1
    if both_watch_count == 0:
        return 0

    rating1 = []
    rating2 = []

    for bookid in train.loc[train.user_id == user1, 'product_id'].to_list():
        if bookid in train.loc[train.user_id == user2, 'product_id'].tolist():
            rating1.append(get_rating(user1, bookid))
            rating2.append(get_rating(user2, bookid))

    if (norm(rating1)*norm(rating2)) == 0:
        return 0
    return dot(rating1, rating2)/(norm(rating1)*norm(rating2))

In [ ]:
cosine_similarity_user(12838006.0, 16635694.0)

0

## Độ đo tương tự Pearson

In [ ]:
def pearson_correlation_user(user1, user2):
    both_watch_count = []
    for bookid in train.loc[(train.user_id == user1), 'product_id'].tolist():
        if bookid in train.loc[(train.user_id == user2), 'product_id'].tolist():
            both_watch_count.append(bookid)
    if len(both_watch_count) == 0:
        return 0
    rating_sum_1 = sum([get_rating(user1, bookid) for bookid in both_watch_count])
    avg_rating_sum_1 = rating_sum_1/len(both_watch_count)

    rating_sum_2 = sum([get_rating(user2, bookid) for bookid in both_watch_count])
    avg_rating_sum_2 = rating_sum_2/len(both_watch_count)

    #Gia tri tren tu
    numerator = sum([(get_rating(user1, bookid)- avg_rating_sum_1)*(get_rating(user2, bookid) - avg_rating_sum_2) for bookid in both_watch_count])
    denominator = sqrt(sum([pow((get_rating(user1, bookid) - avg_rating_sum_1), 2) for bookid in both_watch_count]))*sqrt(sum([pow((get_rating(user2, bookid) - avg_rating_sum_2), 2) for bookid in both_watch_count]))
    if denominator == 0:
        return 0
    return numerator/denominator

In [ ]:
pearson_correlation_user(12838006.0, 16635694.0)

0

# Hàm tính top k user có độ tương đồng 

In [ ]:
def get_similarity_user(userId, numberUser, similarityName):
    userIds = train.user_id.unique().tolist()

    if(similarityName == 'pearson'):
        similarityScoreUser = [(cosine_similarity_user(userId, useri),useri) for useri in userIds[:500] if useri != userId]
    elif(similarityName == 'cosine'):
        similarityScoreUser = [(cosine_similarity_user(userId, useri),useri) for useri in userIds[:500] if useri != userId]

    similarityScoreUser.sort(reverse=True)
    return similarityScoreUser[:numberUser]

In [ ]:
get_similarity_user( 18393598.0	, 10, "pearson")

[(1.0, 27683454.0),
 (1.0, 27018419.0),
 (1.0, 26707397.0),
 (1.0, 26286957.0),
 (1.0, 25710147.0),
 (1.0, 24304082.0),
 (1.0, 23890467.0),
 (1.0, 23096572.0),
 (1.0, 22575012.0),
 (1.0, 21979852.0)]

# Hàm khuyến nghị sách

## Khuyến nghị dựa trên trung bình đánh giá

In [ ]:
def get_recommendation_tbdg_user(userid, number_of_user, metric_similar):
    user_ids = train.user_id.unique().tolist()
    total = {}
    number_rating = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        for bookid in get_bookid(user):
            if bookid not in get_bookid(userid) or get_rating(userid, bookid) == 0:
                if bookid not in total:
                    total[bookid] = 0
                    number_rating[bookid] = 0
                    total[bookid] += get_rating(user, bookid)
                    if get_rating(user, bookid) != 0:
                        number_rating[bookid] +=1
                else:
                    total[bookid] += get_rating(user, bookid)
                    if get_rating(user, bookid) != 0:
                        number_rating[bookid] +=1
                
    ranking = [(to/number_rating[bookid], bookid) for bookid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(bookid, score) for score, bookid in ranking]

    return recomendations

## Khuyến nghị dựa theo trung bình đánh giá có trọng số

In [ ]:
def get_recommendation_tbdgcts_user(userid, number_of_user, metric_similar):
    book_ids = train.user_id.unique().tolist()
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        score = peason
        for bookid in get_bookid(user):
            if bookid not in get_bookid(userid) or get_rating(userid, bookid) == 0:
                if bookid not in total:
                    total[bookid] = 0
                    total[bookid] += get_rating(user, bookid)*score
                    similarity_sum [bookid] = 0
                    similarity_sum [bookid] +=score
                else:
                    total[bookid] += get_rating(user, bookid)*score
                    similarity_sum [bookid] +=score
                
    ranking = [(to/similarity_sum[bookid], bookid) for bookid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(bookid, score) for score, bookid in ranking]

    return recomendations

## Khuyến nghị dựa trên khoảng đánh giá

In [ ]:
def get_recommendation_kdg_user(userid, number_of_user, metric_similar):
    total = {}
    similarity_sum = {}
    List_similar = get_similarity_user(userid, number_of_user, metric_similar)
    for peason, user in List_similar:
        score = peason
        for bookid in get_bookid(user):
            if bookid not in get_bookid(userid) or get_rating(userid, bookid) == 0:
                if bookid not in total:
                    total[bookid] = 0
                    total[bookid] += (get_rating(user, bookid) - get_average_rating_user(user))*score
                    similarity_sum [bookid] = 0
                    similarity_sum [bookid] +=score
                else:
                    total[bookid] += (get_rating(user, bookid) - get_average_rating_user(user))*score
                    similarity_sum [bookid] +=score
                
    ranking = [((get_average_rating_user(userid) + to/similarity_sum[bookid]), bookid) for bookid, to in total.items()]
    ranking.sort()
    ranking.reverse()

    recomendations = [(userid, score) for score, userid in ranking]

    return recomendations

## Khuyến nghị sách

In [ ]:
def get_recommendation_user(userid, number_of_user, metrics_rating, metric_similar):
    if userid not in list(train["user_id"].unique()):
        ranking = [(to, userid) for userid, to in average_rating_bookid.items()]
        ranking.sort()
        ranking.reverse()
        recomendations = [(userid, score) for score, userid in ranking]
        return recomendations
    if (metrics_rating == "tbdg"):
        recomendations = get_recommendation_tbdg_user(userid, number_of_user, metric_similar)
        return recomendations
    elif (metrics_rating == "tbdgcts"):
        recomendations = get_recommendation_tbdgcts_user(userid, number_of_user, metric_similar)
        return recomendations
    elif (metrics_rating == "kdg"):
        recomendations = get_recommendation_kdg_user(userid, number_of_user, metric_similar)
        return recomendations

In [ ]:
# Thử nghiệm
user =  18393598.0	
number_of_user = 10
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
user_recommend = get_recommendation_user(user, number_of_user, metrics_rating, metrics_similar)

In [ ]:
user_recommend 

[(190702874.0, 5.1330049261083746),
 (190702862.0, 5.1330049261083746),
 (190702855.0, 5.1330049261083746),
 (183488718.0, 5.1330049261083746),
 (183488711.0, 5.1330049261083746),
 (183488701.0, 5.1330049261083746),
 (183488700.0, 5.1330049261083746),
 (183488698.0, 5.1330049261083746),
 (181938622.0, 5.1330049261083746),
 (181687303.0, 5.1330049261083746),
 (181170531.0, 5.1330049261083746),
 (175798776.0, 5.1330049261083746),
 (174449880.0, 5.1330049261083746),
 (168293266.0, 5.1330049261083746),
 (168283405.0, 5.1330049261083746),
 (162867684.0, 5.1330049261083746),
 (160779390.0, 5.1330049261083746),
 (160621646.0, 5.1330049261083746),
 (160594650.0, 5.1330049261083746),
 (160488878.0, 5.1330049261083746),
 (160445784.0, 5.1330049261083746),
 (157240665.0, 5.1330049261083746),
 (155878047.0, 5.1330049261083746),
 (154160976.0, 5.1330049261083746),
 (153521908.0, 5.1330049261083746),
 (153521904.0, 5.1330049261083746),
 (153521892.0, 5.1330049261083746),
 (153518828.0, 5.13300492610

# Đánh giá phương pháp

In [ ]:
def get_recommendation_4test_user(X_test, number_of_user, metrics_rating, metric_similar):
    X_test_pred = []
    X_test_userid = X_test['user_id'].tolist()
    X_test_bookid = X_test['product_id'].tolist()
    print("Dự đoán cho tập test với {} điểm dữ liệu".format(len(X_test_userid)))
    for i in range(len(X_test_userid)):
        list_R = get_recommendation_user(X_test_userid[i], number_of_user, metrics_rating, metrics_similar)
        check = 0
        for j in list_R:
            if(X_test_bookid[i] == j[0]):
                X_test_pred.append(j[1])
                check = 1
        if(check == 0 ):
            X_test_pred.append(0)
        print('----------dự đoán cho dòng thứ {}'.format(i), 'là {}'.format(X_test_pred[i]))
    return X_test_pred

## Dự đoán và đánh giá bằng pp KDG và cosine

In [ ]:
import time
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
test3 = test.loc[601:900]

In [ ]:
test3

,id,product_id,user_id,user_name,rating,content,thank_count,rating_average,count
601,7609121,11415525.0,9731550.0,Be Huy,5,NaN,0,4.4,1
602,17590591,57325187.0,9121857.0,Ngô Hưng Hào,5,NaN,0,4.9,1
603,16237908,1020669.0,17676048.0,Nguyễn Diệu Phương Linh,5,NaN,0,4.9,1
604,14874877,106604996.0,642183.0,hait huy,5,"Sách mới, giá tốt.",0,4.8,1
605,8975007,58828012.0,6169005.0,Nguyễn Trần Bảo Khuyên,5,NaN,0,5.0,1
...,...,...,...,...,...,...,...,...,...
896,6758270,65008054.0,8369347.0,Phúc Phan,5,"sách mới, giao hàng nhanh chóng... nếu có shop...",0,4.8,1
897,14002248,49549974.0,22430382.0,Phạm Văn Tùng,5,NaN,0,4.8,1
898,17231245,48372560.0,20163704.0,Yến Hồng,5,Hay,0,4.9,1
899,15555150,23827298.0,7104746.0,Nguyễn Văn Giao,5,sách mới,0,4.6,1


In [29]:
import time
start = time.time()
number_similar = 100
metrics_rating = "kdg" # 1 trong 3 ["tbdg", "tbdgctg", "kdg"]
metrics_similar = "cosine" # 1 trong 2 ["cosine", "peason"]
X_test_pred_rating = get_recommendation_4test_user(test3, number_similar, metrics_rating, metrics_similar)

end = time.time()

print("Tổng thời gian dự đoán :", end - start)

Dự đoán cho tập test với 300 điểm dữ liệu


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


----------dự đoán cho dòng thứ 0 là 4.015602210862349
----------dự đoán cho dòng thứ 1 là 0
----------dự đoán cho dòng thứ 2 là 4.97818707568519
----------dự đoán cho dòng thứ 3 là 4.23849471054546
----------dự đoán cho dòng thứ 4 là 0
----------dự đoán cho dòng thứ 5 là 4.9911647133322266
----------dự đoán cho dòng thứ 6 là 4.84601044894079
----------dự đoán cho dòng thứ 7 là 0
----------dự đoán cho dòng thứ 8 là 4.657894736842105
----------dự đoán cho dòng thứ 9 là 0
----------dự đoán cho dòng thứ 10 là 0
----------dự đoán cho dòng thứ 11 là 0
----------dự đoán cho dòng thứ 12 là 0
----------dự đoán cho dòng thứ 13 là 4.983606557377049
----------dự đoán cho dòng thứ 14 là 5.01123595505618
----------dự đoán cho dòng thứ 15 là 4.598857142857143
----------dự đoán cho dòng thứ 16 là 4.797874125827641
----------dự đoán cho dòng thứ 17 là 5.112244897959184
----------dự đoán cho dòng thứ 18 là 0
----------dự đoán cho dòng thứ 19 là 4.647403163532196
----------dự đoán cho dòng thứ 20 là 0
--

In [30]:
import csv
X_test_pred_rating1= pd.DataFrame(data=X_test_pred_rating)
#X_test_pred_rating1 = X_test_pred_rating
X_test_pred_rating1.to_csv('/content/Mydrive/MyDrive/recomment_system/Do_an/Collaborative_Filtering/CF_User_Based/Pre-KDG-Cosine-601-900.csv',header=True, index = None)


In [ ]:
X_test_pred_rating = list(np.where(np.isnan(X_test_pred_rating), 0, X_test_pred_rating))

X_test_pred_rating_select = []
X_test_true_rating_select = []

for i in range(len(X_test_pred_rating)):
    if (X_test_pred_rating[i] != 0):
        X_test_pred_rating_select.append(X_test_pred_rating[i])
        X_test_true_rating_select.append(test['rating'].tolist()[i])
        X_test_pred_rating_select_round = [np.round(num) for num in X_test_pred_rating_select]

mse = mean_squared_error(X_test_true_rating_select, X_test_pred_rating_select)
rmse = math.sqrt(mse)
mae = mean_absolute_error(X_test_true_rating_select, X_test_pred_rating_select)

X_test_true_rating_select.sort(reverse = True)
max = X_test_true_rating_select[0]
min = X_test_true_rating_select[len(X_test_true_rating_select)-1]
nmae = mae/(max-min)
print('---------------------------------')
print('Đánh giá trên độ đo MSE:', mse)
print('Đánh giá trên độ đo RMSE:', rmse)
print('Đánh giá trên độ đo MAE:', mae)
print('Đánh giá trên độ đo NMAE:', nmae)